# Device Setup

## Import Libraries

The FINNExampleOverlay handles data transfers between the processing system (controlled directly by this notebook) and the programmable logic. It packs the input data into a format the models will expect, then sends and receives data from the programmable logic via DMAs.<br>
get_data contains functions to load test data from .npz files and format it in the required buffer lengths.<br>
Numpy is used for data handling. Platform and pynq contain information about the device being used.

In [ ]:
from driver_base import FINNExampleOverlay
import get_data
import numpy as np
import platform
import pynq

## Get Device Information

Some extra information about the device, required for the overlay.

In [4]:
def get_edge_or_pcie():
    cpu = platform.processor()
    if cpu in ["armv7l", "aarch64"]:
        return "edge"
    elif cpu in ["x86_64"]:
        return "pcie"
    else:
        raise OSError("Platform is not supported.")

In [ ]:
driver_modes = {"edge": "zynq-iodma", "pcie": "alveo"}
target_platform = pynq.Device.active_device.name
driver_mode = driver_modes[get_edge_or_pcie()]

# Run Multi-Layer Perceptron (MLP)

## Get Test Data

The MLP expects a buffer size of ____ and the expected labels are binary values for whether four channels are occupied or not. The data is interwoven so that the iq data is structured in an alternating pattern like [i,q,i,q,i,q,...]. Due to limits with the PYNQ-Z2s storage only ___MB of data is tested with apposed to the available 40GB, but this is still 85800 test cases. 

In [4]:
normalized_array, labels = get_data.get_mlp_data()

In [5]:
print(normalized_array.shape)
print(labels.shape)

(85800, 1, 256)
(85800, 4)


## Initialise the MLP

The bitfile generated in Vivado contains all the information needed to create the MLP in the programmable logic. The io_shape_dict contains any information about how the data should be formatted for inputs and outputs. The overlay uses the bitfile to instanciate the model in hardware, setting up connections with the DMAs to pass data to it. The model can receive sets of data at up to 100MHz, with an output being produced every 16 sets of input.

In [ ]:
bitfile = "bitfile_mlp/finn-accel.bit"
from driver_mlp.driver import io_shape_dict
mlp = FINNExampleOverlay(bitfile, driver_mode, io_shape_dict,fclk_mhz = 100.0)
print(io_shape_dict)

## Test with a Single Input

To test the model receiving and predicting with data a single input is sent to the MLP. The input data is convereted to 8 bit integers as this is what it will receive from the rest of the radio.

In [ ]:
test_single = normalized_array[1].astype(np.int8)
test_single_label = labels[1]

print(test_single.shape)
print(test_single_label)

(1, 256)
[0. 0. 0. 1.]


The overlay provides the execute function to format and send data to the programmable logic. It then waits till it receives an output and returns it.

In [ ]:
mlp.batch_size = 1
accel_out = mlp.execute(test_single)

Since the model output is a bipolar rather than binary it uses -1 to represent no user in a channel rather than 0 like with the labels.

In [9]:
print(accel_out)

[[-1. -1.  1.  1.]]


## Test with All Data

To verify the model is functioning as expected the full available dataset is executed on it. The bipolar outputs are converted to binary and compared with the labels to get the model accuracy.

In [ ]:
#split the input data into 20 equally sized batches
batch_size = 4290
mlp.batch_size = batch_size
(test_imgs, test_labels) = (normalized_array.astype(np.int8),labels)
total = test_imgs.shape[0]
n_batches = total//batch_size
test_imgs = test_imgs.reshape(n_batches,batch_size,256)
test_labels = test_labels.reshape(n_batches,batch_size,4)

#loop through the batches and execute on the MLP
ok = 0
nok = 0
for i in range(n_batches):
    inp = test_imgs[i]
    exp = test_labels[i].astype(np.float32)
    out = mlp.execute(inp)
    out = [(x+1)/2 for x in out.flatten()]
    ok += np.count_nonzero(out == exp.flatten())
    nok += np.count_nonzero(out != exp.flatten())
    acc = 100.0 * ok / (ok+nok)
    print("batch %d / %d : total OK %d NOK %d : accuracy {:.2f}%%" % (i + 1, n_batches, ok, nok,acc))
    
acc = 100.0 * ok / (ok+nok)
print("Final accuracy: {:.2f}%".format(acc))

batch 1 / 20 : total OK 13529 NOK 3631 : accuracy 78.840326%
batch 2 / 20 : total OK 26739 NOK 7581 : accuracy 77.910839%
batch 3 / 20 : total OK 38519 NOK 12961 : accuracy 74.823232%
batch 4 / 20 : total OK 51410 NOK 17230 : accuracy 74.898019%
batch 5 / 20 : total OK 66567 NOK 19233 : accuracy 77.583916%
batch 6 / 20 : total OK 82146 NOK 20814 : accuracy 79.784382%
batch 7 / 20 : total OK 98132 NOK 21988 : accuracy 81.694972%
batch 8 / 20 : total OK 113933 NOK 23347 : accuracy 82.993153%
batch 9 / 20 : total OK 129634 NOK 24806 : accuracy 83.938099%
batch 10 / 20 : total OK 142931 NOK 28669 : accuracy 83.293124%
batch 11 / 20 : total OK 156299 NOK 32461 : accuracy 82.803030%
batch 12 / 20 : total OK 172448 NOK 33472 : accuracy 83.745144%
batch 13 / 20 : total OK 188366 NOK 34714 : accuracy 84.438766%
batch 14 / 20 : total OK 203395 NOK 36845 : accuracy 84.663253%
batch 15 / 20 : total OK 213430 NOK 43970 : accuracy 82.917638%
batch 16 / 20 : total OK 217515 NOK 57045 : accuracy 79.22

# Run Convolutional Neural Network (CNN)

## Get Test Data

The CNN expects a buffer size of ____ and the expected labels are binary values for whether four channels are occupied or not. The iq samples are kept seperate for the CNN, with each being input as 16 by 16 array. Due to limits with the PYNQ-Z2s storage only ___MB of data is tested with apposed to the available 40GB, but this is still ____ test cases. 

In [ ]:
normalized_array, labels = get_data.get_cnn_data()

In [ ]:
print(normalized_array.shape)
print(labels.shape)

(85800, 1, 256)
(85800, 4)


## Initialise the CNN in Hardware

The bitfile generated in Vivado contains all the information needed to create the MLP in the programmable logic. The io_shape_dict contains any information about how the data should be formatted for inputs and outputs. The overlay uses the bitfile to instanciate the model in hardware, setting up connections with the DMAs to pass data to it. The model can receive sets of data at up to 100MHz, with an output being produced every 16 sets of input.

In [ ]:
bitfile = "bitfile_cnn/finn-accel.bit"
from driver_cnn.driver import io_shape_dict
cnn = FINNExampleOverlay(bitfile, driver_mode, io_shape_dict,fclk_mhz = 100.0)

## Test with a Single Input

To test the model receiving and predicting with data a single input is sent to the CNN. The input data is convereted to 8 bit integers as this is what it will receive from the rest of the radio.

In [ ]:
test_single = normalized_array[1].astype(np.int8)
test_single_label = labels[1]

print(test_single.shape)
print(test_single_label)


(1, 16, 16, 2)
[1. 0. 0. 0.]


The overlay provides the execute function to format and send data to the programmable logic. It then waits till it receives an output and returns it.

In [ ]:
cnn.batch_size = 1
accel_out = cnn.execute(test_single)

Since the model output is a bipolar rather than binary it uses -1 to represent no user in a channel rather than 0 like with the labels.

In [10]:
print(accel_out)

[[ 1. -1. -1. -1.]]


## Test with All Data

To verify the model is functioning as expected the full available dataset is executed on it. The bipolar outputs are converted to binary and compared with the labels to get the model accuracy.

In [ ]:
batch_size = 2145
cnn.batch_size = batch_size
(test_imgs, test_labels) = (normalized_array.astype(np.int8),labels)

ok = 0
nok = 0
n_batches = test_imgs.shape[0]//batch_size
total = batch_size*n_batches

test_imgs = test_imgs.reshape(n_batches,batch_size,16,16,2)
test_labels = test_labels.reshape(n_batches,batch_size,4)

for i in range(n_batches):
    inp = test_imgs[i]
    exp = test_labels[i].astype(np.float32)
    out = cnn.execute(inp)
    out = [(x+1)/2 for x in out.flatten()]
    ok += np.count_nonzero(out == exp.flatten())
    nok += np.count_nonzero(out != exp.flatten())
    acc = 100.0 * ok / (ok+nok)
    print("batch %d / %d : total OK %d NOK %d : accuracy %.2f%%" % (i + 1, n_batches, ok, nok,acc))

acc = 100.0 * ok / (ok+nok)
print("Final accuracy: {:.2f}%".format(acc))

batch 1 / 20 : total OK 6919 NOK 1661 : accuracy 80.64%
batch 2 / 20 : total OK 13699 NOK 3461 : accuracy 79.83%
batch 3 / 20 : total OK 20242 NOK 5498 : accuracy 78.64%
batch 4 / 20 : total OK 27434 NOK 6886 : accuracy 79.94%
batch 5 / 20 : total OK 35641 NOK 7259 : accuracy 83.08%
batch 6 / 20 : total OK 44020 NOK 7460 : accuracy 85.51%
batch 7 / 20 : total OK 52551 NOK 7509 : accuracy 87.50%
batch 8 / 20 : total OK 59175 NOK 9465 : accuracy 86.21%
batch 9 / 20 : total OK 65196 NOK 12024 : accuracy 84.43%
batch 10 / 20 : total OK 71300 NOK 14500 : accuracy 83.10%
batch 11 / 20 : total OK 77601 NOK 16779 : accuracy 82.22%
batch 12 / 20 : total OK 86115 NOK 16845 : accuracy 83.64%
batch 13 / 20 : total OK 94522 NOK 17018 : accuracy 84.74%
batch 14 / 20 : total OK 102656 NOK 17464 : accuracy 85.46%
batch 15 / 20 : total OK 108700 NOK 20000 : accuracy 84.46%
batch 16 / 20 : total OK 112225 NOK 25055 : accuracy 81.75%
batch 17 / 20 : total OK 118901 NOK 26959 : accuracy 81.52%
batch 18 / 